In [1]:
import pandas as pd
import numpy as np
import re
from io import StringIO

In [2]:
data = ""
with open("table1.dat.txt", "r") as f:
    for line in f:

        if line[30:33] == "   ":
            line = line[:30] + "---" + line[33:]
        if line[41:44] == "   ":
            line = line[:41] + "---" + line[44:]
        
        line = line[:20].replace(" ", "") + line[20:]
        line = line.replace(" ", ",")
        line = line.replace("|", ",")
 
        for i in line[:15]:
            if i == ",":
                if line[line.index(i)-1] != "," and line[line.index(i)+1] != ",":
                    line = line[:line.index(i)] + " " + line[line.index(i)+1:]
                else:
                    break

        line = re.sub('\,+', ',', line)
            
        data = data+line

In [3]:
labels = ["Host", "mass", "mass_error", "age", "age_error", "nstars", "logPnull", "BIC1-2", "logbetaM20", "logPhigh", "HJ", "Include"]

In [4]:
hosts = pd.read_csv(StringIO(data), sep=",", names=labels)

In [5]:
df = pd.read_csv("exoplanets.csv", skiprows=98)

In [6]:
df["Host"] = df["hostname"].str.replace(" ","")

In [7]:
df = df[["hostname", "Host", "ra", "dec"]]

In [8]:
df1 = pd.merge(hosts, df, how="left", on="Host")

In [9]:
df1.drop_duplicates(subset="Host", inplace=True)

In [10]:
df1.drop(["Host"], axis=1, inplace=True)

In [11]:
df1 = df1.iloc[1:]

In [12]:
df1.dropna(how="any", subset=["hostname"], inplace=True)

In [13]:
df1.set_index("hostname", inplace=True)

In [14]:
df2 = pd.read_csv("gaia_all.csv")

In [15]:
delta = 0.9
vals = np.zeros((df1.shape[0],13))

In [16]:
idx = 0
for i, row in df1.iterrows():
    ra = row["ra"]
    dec = row["dec"]
    #print(row["ra"], row["dec"])
    #print(df2[(df2["ra"] < ra+delta) & (df2["ra"] > ra-delta) & (df2["dec"] < dec+delta) & (df2["dec"] > dec-delta)])
    arr = np.array(df2[(df2["ra"] < ra+delta) & (df2["ra"] > ra-delta) & (df2["dec"] < dec+delta) & (df2["dec"] > dec-delta)].values)
    #print(arr[:, 1])
    a = (np.abs(arr[:, 1] - ra)).argmin()
    vals[idx] = arr[a]
    idx += 1

In [17]:
df3 = pd.DataFrame(vals, columns=df2.columns)

In [18]:
df1.drop(["ra", "dec"], axis=1, inplace=True)
df3.drop(["source_id"], axis=1, inplace=True)

In [19]:
df4 = pd.merge(df1.reset_index(), df3, how="left", on=df1.reset_index().index)

In [20]:
df4.drop(["key_0"], axis=1, inplace=True)

In [21]:
df4.set_index("hostname", inplace=True)

In [22]:
df4.to_csv("hosts.csv")

In [23]:
df2 = df2[(~df2["ra"].isin(df4["ra"]))&(~df2["dec"].isin(df4["dec"]))]

In [24]:
df2.to_csv("gaia.csv")